In [1]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

### Relevante Parameter:

array_phi = 1 / (1 + np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat)))

array_lambdaY = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))

    if r[0] > 0:
        for i in range(len(y_monat)):
            for j in range(y_monat[i] + 1):
                if j == 0:
                    pmf = phi[i] + (1 - phi[i]) * (1 + lambdaY[i] / r[0])**(-r[0])
                elif j > 0:
                    numerator = math.gamma(j + r[0])
                    denominator = math.factorial(j) * math.gamma(r[0])
                    gamma_term = numerator / denominator
                    pmf= (1 - phi[i]) * gamma_term * (1 + lambdaY[i] / r[0])**(-r[0]) * (1 + r[0] / lambdaY[i])**(-j)
                cdf[i] += pmf
    return cdf

Die Öffungszahlen sind mit einer ZINBD modelliert und die Werte folgender Tabellen haben Einfluss:

1. df_LY
2. LO_monat
3. df_trans_werte
4. df_HY

folgende Parameter haben einen Einfluss:
1. delta_0 (1 x anzahl_states array, also state abhängig)
2. delta_1 (1 x anzahl_states array, also state abhängig)

3. alpha_p (1 x anzahl_states array, also state abhängig)
4. beta_p_1 (1 x anzahl_states array, also state abhängig)
5. beta_p_1 (1 x anzahl_states array, also state abhängig)



(1 x 1 array, state unabhängig)

In [2]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY = pd.read_csv(os.path.join(path, filename))

In [3]:
# LO monat
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

In [4]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_trans_werte = pd.read_csv(os.path.join(path, filename))

In [5]:
# EM_monat aus der HY Zahlen der versendeten Mails ziehen
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

### Datenanpassung wie bei der Berechnung der optimalen Parameter

In [6]:
## Alle DataFrame Werte von Float zu int
df_LY = df_LY.astype(int)
df_LO = df_LO.astype(int)
df_trans_werte = df_HY.astype(int)
df_HY = df_LY.astype(int)

### Sicherstellen dass alle df dieselben Kunden verwenden

In [7]:
# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LY = df_LY[df_LY['iid'].isin(df_LO['iid'])]
df_LO = df_LO[df_LO['iid'].isin(df_trans_werte['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_HY['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LY['iid'])]

In [8]:
len(df_LY)

647

In [9]:
len(df_LO)

647

In [10]:
len(df_trans_werte)

697

In [11]:
len(df_HY)

647

In [12]:
# Siherstellen dass alle dataframes die Kunden in derselben Reihenfolge auflisten
df_LY = df_LY.sort_values('iid')
df_LO = df_LO.sort_values('iid')
df_trans_werte = df_trans_werte.sort_values('iid')
df_HY = df_HY.sort_values('iid')

#Index resetten
df_LY = df_LY.reset_index(drop = True)
df_LO = df_LO.reset_index(drop = True)
df_trans_werte = df_trans_werte.reset_index(drop = True)
df_HY = df_HY.reset_index(drop = True)

### Ausreißer bei den Kunden löschen (wie bei den Berechnungen)

In [13]:
kunden_pos = [443, 387, 408, 415, 451, 531, 364, 351, 348, 501, 408, 285, 422, 251, 220]

In [14]:
# Zeilen löschen
df_LY = df_LY.drop(kunden_pos).reset_index(drop = True)
df_LO = df_LO.drop(kunden_pos).reset_index(drop = True)
df_trans_werte = df_trans_werte.drop(kunden_pos).reset_index(drop = True)
df_HY = df_HY.drop(kunden_pos).reset_index(drop = True)

### Spezifikation des Modells

In [15]:
anzahl_kunden = len(df_LY["iid"])
anzahl_kunden

633

In [16]:
anzahl_monate = len(df_LY.columns[df_LY.columns != 'iid'])
anzahl_monate

15

In [17]:
anzahl_states = 2

# Test für 2 states und 200 Iterationen mit bounds

### 1. Herauslesen der Parameter aus dem array der gesamten geschätzten Parameter

In [49]:
estimated_params = [ 0.        , -2.57542179,  0.        ,  2.62717551,  0.        ,
       -2.98703585,  0.        , -2.31989391,  0.        ,  0.1361796 ,
        0.        ,  0.03734605,  0.        ,  2.16428968,  0.        ,
        3.        ,  0.        ,  1.49658634,  0.        , -1.30925976,
        0.53596929,  1.56559093, -1.28623055,  6.88631043,  6.44953524,
       -1.28842799,  1.61663301, -1.12905537, -1.85205332, -0.38670405,
        0.135329  ,  0.15976614, -0.01763291,  0.85690663]

In [50]:
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4

alpha = estimated_params[:alpha_size]
gamma = estimated_params[alpha_size:alpha_size+gamma_size]
alpha_o = estimated_params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
beta_o = estimated_params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
delta_0 = estimated_params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
delta_1 = estimated_params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
alpha_p = estimated_params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
beta_p_1 = estimated_params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
beta_p_2 = estimated_params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
theta = estimated_params[alpha_size+gamma_size+anzahl_states*6+1:]

In [51]:
delta_0

[6.88631043, 6.44953524]

In [52]:
delta_1

[-1.28842799, 1.61663301]

In [53]:
alpha_p

[-1.12905537, -1.85205332]

In [54]:
beta_p_1

[-0.38670405, 0.135329]

In [55]:
beta_p_2

[0.15976614, -0.01763291]

### Erstellen der erwarteten Käufe in den jeweiligen Monaten

### Dispersion Parameter r für die ZINBD Verteilun wurde vorhergeschätzt

In [56]:
r = [1.29]

### Phi und lamdaY Definition für Verwendung in PMF Berechnung

In [57]:
# Definition der Wahrscheinlichkeit des Eintretens eines Erfolges im Einzelversuch phi als anzahl_iids x T df
def phi_state_monat(delta_0, delta_1, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    LY_monat = df_LY[str_monat]
    array_phi = 1 / (1 + np.exp(delta_0[state] + delta_1[state] * np.log(LY_monat)))
    
    return array_phi

# Definition der lambdaY als anzahl_iids x 1 array für monat und state
def lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    
    array_lambdaY = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
    
    return array_lambdaY

### Berechnug PMF

In [58]:
# Array der Verteilungsfunktion der Käufe Y für Monat und state
def PMF_Y_state_monat(value, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    # phi ist das 1 x anzahl_iids array mit phi für jeden kunden mit gegegebenem monat und state
    phi = phi_state_monat(delta_0, delta_1, state, monat)
    # lambdaY ist das 1 x anzahl_iids array mit lammbda für jeden kunden mit gegegebenem monat und state
    lambdaY = lambdaY_state_monat(alpha_p, beta_p_1, beta_p_2, state, monat)
    # pmf als 1x anzahl_iids array mit Nullen initieren
    pmf = np.zeros_like(lambdaY, dtype=float)
    
    if r[0] > 0:
        for i in range(anzahl_kunden):
            if value == 0:
                pmf[i] = phi[i] + (1 - phi[i]) * (1 + lambdaY[i] / r[0])**(-r[0])
            elif value > 0:
                numerator = math.gamma(value + r[0])
                denominator = math.factorial(value) * math.gamma(r[0])
                gamma_term = numerator / denominator
                pmf[i] = (1 - phi[i]) * gamma_term * (1 + lambdaY[i] / r[0])**(-r[0]) * (1 + r[0] / lambdaY[i])**(-(value))
    return pmf

In [59]:
# Wir betrachte die CDF(x) für x in [0, 10]
PMF = np.zeros((11, anzahl_kunden, anzahl_states, anzahl_monate))

In [60]:
for value in range(0, 11):   
        for state in range(1, anzahl_states + 1):
            for monat in range(1, anzahl_monate +1):
                PMF[value, :, state - 1, monat-1] =\
                    PMF_Y_state_monat(value, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)

### Berechnung erwartete Käufe pro Monat, Kunde und state

In [61]:
PMF_monat_kunde_state_erwartung = np.zeros((anzahl_kunden, anzahl_states, anzahl_monate))

In [62]:
for kunde in range(anzahl_kunden):
    for state in range(1, anzahl_states + 1):
            for monat in range(1, anzahl_monate +1):
                    erwartung = 0
                    for value in range(0, 11):
                        erwartung = PMF[value, kunde, state-1, monat-1] * value + erwartung
                    
                    PMF_monat_kunde_state_erwartung[kunde, state-1, monat-1] = erwartung

### Berechnung Erwartungswert der Käufe pro Monat und state

In [63]:
np.sum(PMF_monat_kunde_state_erwartung[:,0,:])/PMF_monat_kunde_state_erwartung[:,0,:].size

0.5848337884626803

In [64]:
np.sum(PMF_monat_kunde_state_erwartung[:,1,:])/PMF_monat_kunde_state_erwartung[:,1,:].size

0.18054875550437158

In [66]:
np.exp(1.566-0.536)

2.801065834699079

In [67]:
0.578 * 2.801065834699079

1.6190160524560675

In [69]:
np.exp(-1.286)

0.27637407132753633

# Random Tests

In [48]:
1/(1+np.exp(6.450+1.617*np.log(1)))

0.001578028060403861